In [18]:
import pandas as pd

In [19]:
df=pd.read_csv("car_ownership.csv")
df.head()

monthly_salary  owns_car
0           22000         0
1           25000         0
2           47000         1
3           52000         0
4           46000         1

In [40]:
X=df[["monthly_salary"]]
y=df.owns_car
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

params={
    "random_state":42,
    'solver':'lbfgs',
    'max_iter':1000,
    'n_jobs':5}
    
model=LogisticRegression(**params)
model.fit(X_train,y_train)

LogisticRegression(max_iter=1000, n_jobs=5, random_state=42)

In [41]:
model.score(X_test,y_test)

0.8888888888888888

In [42]:
y_pred=model.predict(X_test)
y_pred

array([1, 0, 1, 0, 0, 0, 0, 1, 0])

#### classification precision and recall

In [54]:
from sklearn.metrics import classification_report
report=classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.83      1.00      0.91         5
           1       1.00      0.75      0.86         4

    accuracy                           0.89         9
   macro avg       0.92      0.88      0.88         9
weighted avg       0.91      0.89      0.89         9



In [55]:
from sklearn.metrics import classification_report
report_dict=classification_report(y_test,y_pred,output_dict=True)
print(report_dict)

{'0': {'precision': 0.8333333333333334, 'recall': 1.0, 'f1-score': 0.9090909090909091, 'support': 5.0}, '1': {'precision': 1.0, 'recall': 0.75, 'f1-score': 0.8571428571428571, 'support': 4.0}, 'accuracy': 0.8888888888888888, 'macro avg': {'precision': 0.9166666666666667, 'recall': 0.875, 'f1-score': 0.8831168831168831, 'support': 9.0}, 'weighted avg': {'precision': 0.9074074074074076, 'recall': 0.8888888888888888, 'f1-score': 0.886002886002886, 'support': 9.0}}


In [30]:
 import mlflow

## logging into ml flow

In [58]:
mlflow.set_experiment("First experiment")
mlflow.set_tracking_uri("http://127.0.0.1:5000")
with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metrics({
        'accuracy':report_dict['accuracy'],
        'recall_0':report_dict['0']['recall'],
        'recall_1':report_dict['1']['recall']
    })
    mlflow.sklearn.log_model(model,"Logistic Regressio")
    

2024/10/31 15:17:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/31 15:17:33 INFO mlflow.tracking._tracking_service.client: 🏃 View run traveling-ox-636 at: http://127.0.0.1:5000/#/experiments/399819398117135024/runs/5af6bb7a01764572806d7f278bf1db04.
2024/10/31 15:17:33 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/399819398117135024.


In [60]:
mlflow.set_experiment("second experiment")
mlflow.set_tracking_uri("http://127.0.0.1:5000")
with mlflow.start_run(run_name='logistic regression'):
    mlflow.log_params(params)
    mlflow.log_metrics({
        'accuracy':report_dict['accuracy'],
        'recall_0':report_dict['0']['recall'],
        'recall_1':report_dict['1']['recall']
    })
    mlflow.sklearn.log_model(model,"Logistic Regressio")
    

2024/10/31 17:34:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/31 17:34:34 INFO mlflow.tracking._tracking_service.client: 🏃 View run logistic regression at: http://127.0.0.1:5000/#/experiments/163735128155891726/runs/8ddf2e8562104c7e88cdc4cf600b6498.
2024/10/31 17:34:34 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/163735128155891726.


## Register the model into the mlflow

In [64]:
# Register the model with a name in the Model Registry
model_name = "Logistic Regression"
run_id=input("Enter runid of the version of the model you want to register") ##you can find the run id in the mlflow ui at which version of model you want to load 
model_uri=f"runs:/{run_id}/{model_name}"
# This registers the model; if a model with the name exists, it will create a new version
model_version = mlflow.register_model(model_uri=model_uri, name=model_name)

print(f"Model registered with name: {model_name} and version: {model_version.version}")


Enter runid of the version of the model you want to register 8ddf2e8562104c7e88cdc4cf600b6498


Registered model 'Logistic Regression' already exists. Creating a new version of this model...
2024/10/31 17:46:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Logistic Regression, version 2


Model registered with name: Logistic Regression and version: 2


Created version '2' of model 'Logistic Regression'.


## loading the model into MLFLOW

In [66]:
run_id=input("Enter runid of the version of the model you want to load") ##you can find the run id in the mlflow ui at which version of model you want to load 
loaded_model = mlflow.sklearn.load_model(f"runs:/{run_id}/{model_name") 
#@challenger is a alias tag given to the registered model's specific version in mlflow 
# you can give any tag you want , usually the final product model given as @champion

Enter runid of the version of the model you want to load 8ddf2e8562104c7e88cdc4cf600b6498


MlflowException: The following failures occurred while downloading one or more artifacts from http://127.0.0.1:5000/api/2.0/mlflow-artifacts/artifacts/163735128155891726/8ddf2e8562104c7e88cdc4cf600b6498/artifacts:
##### File Logistic Regression #####
API request to http://127.0.0.1:5000/api/2.0/mlflow-artifacts/artifacts/163735128155891726/8ddf2e8562104c7e88cdc4cf600b6498/artifacts/Logistic Regression failed with exception HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/163735128155891726/8ddf2e8562104c7e88cdc4cf600b6498/artifacts/Logistic%20Regression (Caused by ResponseError('too many 500 error responses'))